In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
import kerastuner as kt

C:\Users\SB15\AppData\Local\Temp\ipykernel_30244\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
img_train = img_train.astype("float32") / 255.0
img_test = img_test.astype("float32") / 255.0

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int("uints", min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units = hp_units, activation="relu"))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice("learning_rate", values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

    return model

In [6]:
tuner = kt.Hyperband(model_builder, objective="val_accuracy", max_epochs=10, factor=3, directory="train_result", project_name="intro_to_kt")

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data=(img_test, label_test), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 43s]
val_accuracy: 0.8503999710083008

Best val_accuracy So Far: 0.887499988079071
Total elapsed time: 00h 08m 25s
INFO:tensorflow:Oracle triggered exit


In [9]:
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs=10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4785 - accuracy: 0.8297 - val_loss: 0.4125 - val_accuracy: 0.8507
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3609 - accuracy: 0.8681 - val_loss: 0.4312 - val_accuracy: 0.8481
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3224 - accuracy: 0.8809 - val_loss: 0.3660 - val_accuracy: 0.8710
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3012 - accuracy: 0.8880 - val_loss: 0.3762 - val_accuracy: 0.8706
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2813 - accuracy: 0.8956 - val_loss: 0.3771 - val_accuracy: 0.8651
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2670 - accuracy: 0.9007 - val_loss: 0.3535 - val_accuracy: 0.8763
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2540 - accuracy: 0.9050 - val_loss: 0.3311 - val_accuracy: